In [1]:
using TSSM
include("time_propagators.jl")

 *** OPENMP n_threads =           8


step! (generic function with 6 methods)

# Test problem: nonlinear Schrödinger equation with time dependent potential

The soliton
$$\psi_\mathrm{ex}(x,t)=\frac{a\mathrm{e}^{\frac{1}{2}\mathrm{i}((a^2-b^2)t-bx)}}{\cosh(a(bt+x-c))}$$ 
is the exact solution of the cubic nonlinear Schrödinger
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+\kappa|\psi(x,t)|^2\psi(x,t),
\quad \psi(x,0) = \psi_\mathrm{ex}(x,0),$$
where $\kappa=-1$, and $a,b,c\in\mathbb{R}$ are arbitrarily chosen parameters. 

It follows that 
the exact solution of the *nonlinear* Schrödinger equation
$$\mathrm{i}\frac{\partial}{\partial t}\psi(x,t)=-\frac{1}{2}\Delta\psi(x,t)+V_\alpha(x,t)\psi(x,t)+\alpha\kappa|\psi(x,t)|^2\psi(x,t),\quad\psi(x,0) = \psi_\mathrm{ex}(x,0),$$
with *time dependent potential*
$$V_\alpha(x,t)= (1-\alpha)\kappa|\psi_\mathrm{ex}(x,t)|^2=\frac{(1-\alpha)\kappa a^2}{\cosh(a(bt+x-c))^2}$$
and, e.g., $\alpha=\frac{1}{2}$ is again given by $\psi_\mathrm{ex}(x,t)$.

We use this time-dependent problem to verify the correct implementation of various time propagation methods

In [2]:
# exact solution
function soliton(x, t)
    a = 2.0
    b = 1.0
    c = 0.0    
    h = (a^2 - b^2)/2*t - b*x
    (a./cosh(a*(b*t+x-c))).*exp(1im*h) 
end    

soliton (generic function with 1 method)

In [3]:
nx = 1024
xmin = -16
xmax = +16
V(x,t) = -0.5*abs(soliton(x,t)).^2
m = Schroedinger1D(nx, xmin, xmax, potential_t=V, cubic_coupling=-0.5)
psi = wave_function(m)
psi_ref = wave_function(m)

┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = cfunction_check_return_type(::Function, ::Type, ::Tuple{DataType}) at TSSM.jl:496
└ @ TSSM /home/alexgrosz/.julia/dev/TSSM/src/TSSM.jl:496
┌ Warning: The function `cfunction` is now written as a macro `@cfunction`.
│   caller = cfunction(::Function, ::Type, ::Tuple{DataType}) at deprecated.jl:55
└ @ Base ./deprecated.jl:55
┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = cfunction_check_return_type(::Function, ::Type, ::Tuple{DataType,DataType}) at TSSM.jl:496
└ @ TSSM /home/alexgrosz/.julia/dev/TSSM/src/TSSM.jl:496
┌ Warning: The function `cfunction` is now written as a macro `@cfunction`.
│   caller = cfunction(::Function, ::Type, ::Tuple{DataType,DataType}) at deprecated.jl:55
└ @ Base ./deprecated.jl:55


WfSchroedinger1D{Float64}(Ptr{Nothing} @0x0000000002a7ee00, Schroedinger1D{Float64}(Ptr{Nothing} @0x0000000002a7bc70))

In [4]:
t0 = 0.0
tend = 1.0
set!(psi_ref, soliton, tend) # exact reference solution

┌ Warning: `cfunction(f, r, a::Tuple)` is deprecated, use `cfunction(f, r, Tuple{a...})` instead.
│   caller = set!(::WfSchroedinger1D{Float64}, ::Function, ::Float64) at tssm_common.jl:415
└ @ TSSM /home/alexgrosz/.julia/dev/TSSM/src/tssm_common.jl:415


# Multistep methods

In [5]:
mutable struct ExactSolution <: TimePropagationMethod
end
function step!(m::ExactSolution, psi::WaveFunction, 
         t0::Real, dt::Real, steps::Int, step::Int)
    set!(psi, soliton, t0+(step+1)*dt)
end    

step! (generic function with 7 methods)

## Exponential multistep
### exact evaluation of the integral; 4 steps; exact starting values; no correction step

In [6]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=0, starting_method=ExactSolution())
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.888e-01                               4
  2   1.250e-01   6.979e-02   2.397e+01   2.81            8
  3   6.250e-02   4.556e-03   2.508e+02   3.94           16
  4   3.125e-02   2.948e-04   2.600e+02   3.95           32
  5   1.563e-02   1.849e-05   3.036e+02   3.99           64
  6   7.813e-03   1.153e-06   3.142e+02   4.00          128
  7   3.906e-03   7.193e-08   3.140e+02   4.00          256
  8   1.953e-03   4.490e-09   3.121e+02   4.00          512
  9   9.766e-04   2.804e-10   3.110e+02   4.00         1024
 10   4.883e-04   1.742e-11   3.262e+02   4.01         2048


subdivision for exact solution is consistent:

In [7]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=0, starting_method=ExactSolution(), starting_subdivision=5)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.888e-01                               4
  2   1.250e-01   6.979e-02   2.397e+01   2.81            8
  3   6.250e-02   4.556e-03   2.508e+02   3.94           16
  4   3.125e-02   2.948e-04   2.600e+02   3.95           32
  5   1.563e-02   1.849e-05   3.036e+02   3.99           64
  6   7.813e-03   1.153e-06   3.142e+02   4.00          128
  7   3.906e-03   7.193e-08   3.140e+02   4.00          256
  8   1.953e-03   4.490e-09   3.121e+02   4.00          512
  9   9.766e-04   2.804e-10   3.110e+02   4.00         1024
 10   4.883e-04   1.742e-11   3.262e+02   4.01         2048


### Exponential multistep version 2; 5 steps; starting values by Suzuki composition method; one correction step

In [8]:
set!(psi, soliton, t0)
g = [1/(4-4^(1/3)),1/(4-4^(1/3)),-4^(1/3)/(4-4^(1/3)), 1/(4-4^(1/3)), 1/(4-4^(1/3))] # Suzuki
a, b = get_coeffs_composition(g)
#starting_method = SplittingMethod(a,b)
starting_method = CompositionMethod(g,3)
method = ExponentialMultistep(5, iters=1, version=2, starting_method=starting_method)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   4.153e-02                              84
  2   1.250e-01   1.057e-01   6.411e-03  -1.35           92
  3   6.250e-02   1.422e-02   4.348e+01   2.89          108
  4   3.125e-02   9.562e-04   6.944e+02   3.89          140
  5   1.563e-02   2.730e-05   5.044e+04   5.13          204
  6   7.813e-03   7.407e-07   6.837e+04   5.20          332
  7   3.906e-03   1.416e-08   7.927e+05   5.71          588
  8   1.953e-03   2.227e-10   3.785e+06   5.99         1100
  9   9.766e-04   3.491e-12   3.894e+06   6.00         2124
 10   4.883e-04   3.389e-13   4.701e-02   3.36         4172


Starting values of used composition method seem to be accurate enough for 5 steps, subdivision does not deteriorate results

In [9]:
method = ExponentialMultistep(5, iters=1, version=2, starting_method=starting_method, starting_subdivision=5)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   1.387e-04                             404
  2   1.250e-01   1.060e-01   2.372e-10  -9.58          412
  3   6.250e-02   1.422e-02   4.394e+01   2.90          428
  4   3.125e-02   9.559e-04   6.968e+02   3.90          460
  5   1.563e-02   2.733e-05   5.004e+04   5.13          524
  6   7.813e-03   7.423e-07   6.809e+04   5.20          652
  7   3.906e-03   1.416e-08   8.092e+05   5.71          908
  8   1.953e-03   2.214e-10   3.948e+06   6.00         1420
  9   9.766e-04   3.435e-12   4.255e+06   6.01         2444
 10   4.883e-04   3.514e-13   2.736e-02   3.29         4492


### Exponential multistep version 2; 4 steps; starting values by RK4 for example purposes

In [10]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=1, version=2, starting_method=ExponentialRungeKutta(:rk4))
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01         NaN                              17
  2   1.250e-01         NaN         NaN    NaN           25
  3   6.250e-02         NaN         NaN    NaN           41
  4   3.125e-02         NaN         NaN    NaN           73
  5   1.563e-02         NaN         NaN    NaN          137
  6   7.813e-03   3.873e-01         NaN    NaN          265
  7   3.906e-03   9.206e-05   9.028e+24  12.04          521
  8   1.953e-03   2.067e-08   1.423e+25  12.12         1033
  9   9.766e-04   4.526e-11   1.790e+16   8.84         2057
 10   4.883e-04   1.594e-12   1.537e+04   4.83         4105


Degenerated results because of instable RK4 values can be (partially) repaired using high subdivision for starting values

In [11]:
set!(psi, soliton, t0)
method = ExponentialMultistep(4, iters=1, version=2, starting_method=ExponentialRungeKutta(:rk4), starting_subdivision=100)
global_orders(method, psi, psi_ref, t0, tend, .25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01         NaN                            1205
  2   1.250e-01         NaN         NaN    NaN         1213
  3   6.250e-02         NaN         NaN    NaN         1229
  4   3.125e-02   1.326e-03         NaN    NaN         1261
  5   1.563e-02   5.238e-05   1.378e+04   4.66         1325
  6   7.813e-03   1.199e-06   3.635e+05   5.45         1453
  7   3.906e-03   3.699e-08   4.520e+04   5.02         1709
  8   1.953e-03   1.333e-09   1.300e+04   4.79         2221
  9   9.766e-04   4.539e-11   2.168e+04   4.88         3245
 10   4.883e-04   1.601e-12   1.524e+04   4.83         5293


### Exponential multistep version 2; 8 steps, starting values by Krogstad

In [12]:
set!(psi, soliton, t0)
method = ExponentialMultistep(10, iters=1, version=2, starting_method=ExponentialRungeKutta(:krogstad))
global_orders(method, psi, psi_ref, t0, tend, 0.25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   2.939e+00                              45
  2   1.250e-01   4.673e-01   1.163e+02   2.65           45
  3   6.250e-02   9.998e-02   4.769e+01   2.22           59
  4   3.125e-02   5.596e-03   1.019e+04   4.16           91
  5   1.563e-02   1.093e-04   1.971e+06   5.68          155
  6   7.813e-03   6.545e-07   2.367e+09   7.38          283
  7   3.906e-03   1.297e-09   5.447e+12   8.98          539
  8   1.953e-03   1.490e-11   4.283e+06   6.44         1051
  9   9.766e-04   5.191e-13   1.968e+02   4.84         2075
 10   4.883e-04   3.258e-13   5.483e-11   0.67         4123


Subdivision is beneficial with high order multistep methods:

In [13]:
set!(psi, soliton, t0)
method = ExponentialMultistep(10, iters=1, version=2, starting_method=ExponentialRungeKutta(:krogstad), starting_subdivision=2)
global_orders(method, psi, psi_ref, t0, tend, 0.25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   2.993e+00                              81
  2   1.250e-01   4.704e-01   1.212e+02   2.67           81
  3   6.250e-02   1.000e-01   4.895e+01   2.23           95
  4   3.125e-02   5.596e-03   1.020e+04   4.16          127
  5   1.563e-02   1.093e-04   1.972e+06   5.68          191
  6   7.813e-03   6.544e-07   2.369e+09   7.38          319
  7   3.906e-03   1.225e-09   8.135e+12   9.06          575
  8   1.953e-03   1.425e-12   3.640e+14   9.75         1087
  9   9.766e-04   2.417e-13   1.229e-05   2.56         2111
 10   4.883e-04   3.223e-13   1.357e-14  -0.42         4159


In [14]:
set!(psi, soliton, t0)
method = ExponentialMultistep(10, iters=1, version=2, starting_method=ExponentialRungeKutta(:krogstad), starting_subdivision=3)
global_orders(method, psi, psi_ref, t0, tend, 0.25, rows=10)

             dt         err           C      p      B calls
-----------------------------------------------------------
  1   2.500e-01   2.997e+00                             117
  2   1.250e-01   4.706e-01   1.216e+02   2.67          117
  3   6.250e-02   1.000e-01   4.903e+01   2.23          131
  4   3.125e-02   5.596e-03   1.020e+04   4.16          163
  5   1.563e-02   1.093e-04   1.972e+06   5.68          227
  6   7.813e-03   6.544e-07   2.369e+09   7.38          355
  7   3.906e-03   1.224e-09   8.144e+12   9.06          611
  8   1.953e-03   1.108e-12   2.728e+15  10.11         1123
  9   9.766e-04   2.435e-13   9.251e-07   2.19         2147
 10   4.883e-04   3.230e-13   1.443e-14  -0.41         4195
